.

Data_Engineering_TIL(20201026)

### [실습 시 참고자료]

블로그글 'Running Scikit-learn models in Apache Pyspark'을 읽고 실습한 내용입니다.

** URL : https://medium.com/@the.data.yoga/running-scikit-models-in-apache-pyspark-4a2ac5e693c4


### [실습전 참고사항]

1) Spark Shuffle 이란

Shuffling is a mechanism Spark uses to redistribute the data across different executors and even across machines. Spark shuffling triggers when we perform certain transformation operations like gropByKey(), reducebyKey(), join() on RDD and DataFrame.

Spark Shuffle is an expensive operation since it involves the following

- Disk I/O


- Involves data serialization and deserialization


- Network I/O

When creating an RDD, Spark doesn't necessarily store the data for all keys in a partition since at the time of creation there is no way we can set the key for data set.

2) spark broadcast 이란

In Spark RDD and DataFrame, Broadcast variables are read-only shared variables that are cached and available on all nodes in a cluster in-order to access or use by the tasks. Instead of sending this data along with every task, spark distributes broadcast variables to the machine using efficient broadcast algorithms to reduce communication costs.


### [실습목표]


Scikit-learn model을 Pyspark 엔진을 이용해서 prediction 해본다.


### [실습내용]

#### step 1) training model with sklearn

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn import datasets

# import some data to play with
iris = datasets.load_iris()
X = iris.data[:, :2]  # we only take the first two features.
Y = iris.target

logreg = LogisticRegression(C=1e5)

# Create an instance of Logistic Regression Classifier and fit the data.
logreg.fit(X, Y)
#prediction
logreg.predict(X[0:1])

#### step 2) save model to local

In [ ]:
import pickle
pickle.dump(logreg, open( "model save path", "wb" ) )

#### step 3) make pyspark wrapper

In [ ]:
def predictor(s_l, s_w, p_l, p_w):
    '''
    accepts the four features as parameters and returns the predicted score as output
    '''
    
    #open picked model
    serialized_model = open("model save path", "rb")
    model = pickle.load(serialized_model)
    serialized_model.close()
    
    #call predict method for model
    return model.predict([s_l, s_w, p_l, p_w])

#### step 4) Turning the python function to a pyspark UDF

Option 1 (Simply registering python function to pyspark as UDF )

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

udf_predictor = udf(predictor, FloatType())

#apply the udf to dataframe
df_prediction = df.withColumn("prediction",udf_predictor(df.sepal_length,df.sepal_width,df.petal_length,df.petal_width))

Option 2(Broadcast the model to spark executors and predict)

In order to remove the inefficiency in option 1, this technique allows reading the model once from the disk and send(broadcast) the model to all spark executors.

In [ ]:
#open picked model
serialized_model = open("model save path", "rb")
model = pickle.load(serialized_model)
serialized_model.close()

#broadcast model to spark executors using spark context(sc)
sc.broadcast(model)

#update prediction method
def predictor(s_l, s_w, p_l, p_w):
    #call predict method for model
    return model.predict([s_l, s_w, p_l, p_w])

#register python method as spark UDF and call over dataframe
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

udf_predictor = udf(predictor, FloatType())

#apply the udf to dataframe
df_prediction = df.withColumn("prediction",udf_predictor(df.sepal_length,df.sepal_width, df.petal_length, df.petal_width))